# Selin's notebook

In this notebook I add the functions that Selin needs to complete her MSc research.

In [2]:
from QG_functions import *

import numpy as np
import pandas as pd

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.periodic_table import Element
from pymatgen.io.cif import *

from ase.visualize import view


from pymatgen.io.ase import AseAtomsAdaptor
import sys

import re
import shutil as sh
import pickle
from tqdm import tqdm


import copy
from sklearn.metrics import mean_squared_error 

#import dataframe_image as dfi

from scipy import constants
from scipy.spatial import KDTree, distance_matrix

import matplotlib.pyplot as plt

import itertools
from itertools import chain

from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error as mse


k_b = constants.physical_constants['Boltzmann constant in eV/K'][0]
# print(k_b)
def vview(structure):
    view(AseAtomsAdaptor().get_atoms(structure))

np.seterr(divide='ignore')
plt.style.use('tableau-colorblind10')

import seaborn as sns
import time

## Read the initial structures from cif

Task: familiarise yourself with the pymatgen Structure object (eg., caf2 and delithiated_structure_init). What are its functions and attributes? Type "caf2." in a code cell to get a prompt of all possible ones. In particluar, check out how to:
- list the atomic number and coordinates
- generate the distance matrix
- select the elements of the distance matrix that correspond to a certain atom type interaction (you will need to use np.where())

Compare the fully lithaited and delithiated cells and volumes.


In [12]:
caf2 = Structure.from_file('data/structures/CaF2.cif')
vview(caf2)

In [13]:
fully_lithiated_structure_init = Structure.from_file('data/fully_lithiated_tmp.cif')
delithiated_structure_init = Structure.from_file('data/delithiated_tmp.cif')

In [ ]:
n_sites = fully_lithiated_structure_init.num_sites
fully_lithiated_structure_init.translate_sites(np.arange(n_sites),[1,1,1],to_unit_cell=True) #what is happening here?

vview(delithiated_structure_init)

## Read the structures from the database

Understand how to read an entry from a database. What type of object is "data"? Try (type(data)). How do you acces information stored into data and within each entry stored into data? 

In [19]:
with open('data/database/config_size_1.pkl', 'rb') as file:
    data = pickle.load(file)

db_entry_example = data[list(data.keys())[0]]
# Print the loaded data to verify
data[list(data.keys())[0]]

{'initial': {'energy': -2947.30808963,
  'energy_ip': 499.06909695,
  'energy_long': -3446.37718658,
  'energy_long_real': -2135.94536095,
  'energy_long_imag': -1310.43182563,
  'lattice_vectors': [[8.872874, 0.0, 0.0],
   [0.0, 9.432501, 0.0],
   [0.0, 0.0, 8.565761]],
  'config': [['Tc', 'core', 0.620302, 0.243497, 0.083331],
   ['Mn', 'core', 0.620302, 0.743492, 0.750012],
   ['Mn', 'core', 0.1203, 0.506495, 0.416677],
   ['Mn', 'core', 0.837392, 0.493495, 0.250004],
   ['Mn', 'core', 0.620302, 0.743492, 0.083331],
   ['Mn', 'core', 0.620302, 0.243497, 0.416677],
   ['Mn', 'core', 0.337391, 0.756492, 0.916685],
   ['Mn', 'core', 0.837392, 0.493495, 0.916685],
   ['Mn', 'core', 0.837392, 0.993489, 0.916685],
   ['Mn', 'core', 0.837392, 0.493495, 0.583339],
   ['Mn', 'core', 0.1203, 0.0065, 0.416677],
   ['Mn', 'core', 0.337391, 0.256497, 0.583339],
   ['Mn', 'core', 0.337391, 0.756492, 0.583339],
   ['Mn', 'core', 0.620302, 0.243497, 0.750012],
   ['Mn', 'core', 0.837392, 0.993489, 

### Read a database entry into a pymatgen object

Print the distance between the Li and Tc atom (Tc is used instead of Mn3+)

In [20]:
structure_example = db_to_structure(data[list(data.keys())[0]])
vview(structure_example)

2025-04-17 08:58:46.637 python[13148:29377314] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-17 08:58:46.637 python[13148:29377314] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Analyse Mn optimised positions

Here we want to see how much the Mn atoms move from their initial positions. Check what the code below is doing. What metric can we calculate on mn_coords_all to see what is the average displacement? What final coordinates for Mn should we use? Do these differ from low concentrations to high concentrations?

In [28]:
mn_coords_all = []
num_samples = 100 #MAKE RANDOM

for i in tqdm(range(1,25), desc="Computing Mn positions"):
    with open(f'data/database/config_size_{i}.pkl', 'rb') as file:
        data = pickle.load(file)
    samples = np.random.choice(len(data),num_samples)

    structure_n = np.array(list(data.keys()))[samples]
    mn_coords_all = []
    mn_coords_diff_all = []
    for j in structure_n:
        coords_tmp = np.array(data[j]['initial']['config'])
        mn_coords_init = np.array(coords_tmp[0:24][:,2:]) 
        mn_coords_init = np.array(mn_coords_init,dtype='float')
        
        ordering = np.lexsort(mn_coords_init.T)
        mn_coords_init_ordered = mn_coords_init[ordering]

        coords_tmp = np.array(data[j]['final']['config'])#[-48:])[:,2:]
        mn_coords = np.array(coords_tmp[0:24][:,2:])

        mn_coords = np.array(mn_coords,dtype='float')
        mn_coords_ordered = np.array(mn_coords,dtype='float')[ordering]
        
        mn_coords_diff = mn_coords_ordered - mn_coords_init

        mn_coords_all.append(mn_coords_ordered.tolist())
        mn_coords_diff_all.append(mn_coords_diff.tolist())
        

    mn_coords_all = np.array(mn_coords_all)
    mn_coords_diff_all = np.array(mn_coords_diff_all)

Computing Mn positions: 100%|██████████| 24/24 [00:02<00:00, 11.00it/s]


## Analyse O optimised positions
Write the code to do the same analysis as Mn.

## Build the Li position grid
Coming soon